In [1]:
# Final merger time for the reference systems with masses m_1 = 1e5 M_solar
# and m_2 = 1 M_solar, as a function of the initial 
# angular momentum j0 for binaries having Power Law PDF (1-1e5) for PBHs having alpha-disk 
# with dynamical friction (Ostriker) as dissipative force.

%matplotlib inline
import numpy as np
from scipy.integrate import odeint
import math
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatterExponent
from matplotlib.colors import LogNorm
from matplotlib import ticker, cm
from accretion import *
import matplotlib as mpl
from matplotlib.lines import Line2D
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.special import gamma
import mass_function
from imripy import halo, constants as c, merger_system as ms, inspiral, waveform, detector, plot_utils
from imripy.inspiral import forces
import imripy.constants as c

plt.rcParams.update({
  "text.usetex": True,
  "font.family": "serif"
})


π = np.pi
Ω_cdm = 0.85
G = 4.4959e-15            #in units of M☉^-1 pc^3 yr^-2
c = 0.3068                #in units of pc yr^-1
ρ_eq = 3.1808e3           #in units of M☉ pc^-3 with ρ_eq=2.1548e-16 kg m^-3
pc = 3.0857e16            # in meters
yr = 3.154e7              # in units of seconds
t_eq = 1.5923e12/yr       # in units of seconds
t_m = 13.78e9             #in units of yrs corresponding to t_0=13.78Gyr
t_0 = 13.78e9             #in units of yrs corresponding to t_0=13.78Gyr
ρ_m = 4e19                #ρ_m = 4e19 M☉ Gpc^-3
M_solar = 1.989e30        # in units of kg
σ_eq = 0.005

solar_mass_to_pc = 4.8e-14
year_to_pc = 0.3064


# The initial value of a_i or a_i_ref is calculated for a reference
# binary with PBHs of masses m_i = 1 solar mass and m_i = 1e-4 solar mass 
# (chosen by choice).
bin_centres = np.geomspace(1, 1e5, 5)
bin_edges = np.sqrt(bin_centres[:-1]*bin_centres[1:])


ratio = bin_edges[1]/bin_edges[0] 
bin_edges = np.append(bin_edges[0]/ratio, bin_edges) 
bin_edges = np.append(bin_edges, bin_edges[-1]*ratio) 


deltas = np.diff(bin_edges)
Δ_1_list = deltas.tolist()
Δ_2_list = deltas.tolist()

m_1 = np.geomspace(1, 1e5, 5)
m_2 = np.geomspace(1, 1e5, 5)
m_1_list = m_1.tolist()
m_2_list = m_2.tolist()   


    
def a(m_1,m_2,Δ_1,Δ_2):  # for x = x_bar 
    
    f_pbh= 0.1
    f = 0.85 * f_pbh
    def P(m): #Powerlaw PBH mass distribution
        α = 1.6
        M = 1    #in units of M☉, for PBHs mass range of 1 M☉ - 1e5 M☉ .
        return ((α-1)/M) * ((m/M)**(-α))

    def f_(m):
        return f*P(m)
    
    def f_b(m_1,m_2):
        return  f_(m_1)+f_(m_2)
    
    def x̄(m_1,m_2,Δ_1,Δ_2):
        return (((3*(m_1+m_2))/(4*π*ρ_eq*f_b(m_1,m_2)*np.sqrt(Δ_1*Δ_2)))**(1/3))
   
    def λ(m_1,m_2,Δ_1,Δ_2):
        return (4*π*ρ_eq*(x̄(m_1,m_2,Δ_1,Δ_2)**3))/(3*(m_1 + m_2))

    
    return (0.0977*λ(m_1,m_2,Δ_1,Δ_2) + 0.0068*(λ(m_1,m_2,Δ_1,Δ_2)**2) ) * x̄(m_1,m_2,Δ_1,Δ_2)





a_i = a(m_1_list[-1], m_2_list[0], Δ_1_list[-1], Δ_2_list[0])
print("a_i = ", a_i, "pc")


# Setting initial and final conditions of the binary evolution for an eccentric orbit
m1 =  1e5 * solar_mass_to_pc  
m2 = 1 * solar_mass_to_pc  
D = 0
accuracy = 1e-13
alpha = 0.1
f_edd = 0.1
eps = 0.1
sp_0 = ms.SystemProp(m1, m2, halo.ConstHalo(0.), D)
alphaDisk = halo.AlphaDisk(m1, alpha, f_edd, eps)
sp_b_alpha = ms.SystemProp(m1, m2, halo.ConstHalo(0.), baryonicHalo = alphaDisk, D=D)
opt_DF = inspiral.Classic.EvolutionOptions(dissipativeForces={forces.GWLoss(), forces.GasDynamicalFriction()}, 
                                         considerRelativeVelocities=True, progradeRotation=False, accuracy=accuracy, verbose=1)



a0 = a_i * 0.1
j0 = 1e-3
e0 = np.sqrt(1 -(j0**2))
R_fin = sp_0.r_isco()   #Choosen equal to r_icso, The final condition for the evolution



m1_ref1 = 1e-2          # in units of solar mass
m1_ref2 = 1             # in units of solar mass
T = 1e-6 * (t_m * year_to_pc ) * ((m1_ref1/m1_ref2)**(-1))

ev_alpha_DF = inspiral.Classic.Evolve(sp_b_alpha, a0, e0, a_fin = R_fin, t_fin = T, opt=opt_DF)
t_final = ev_alpha_DF.t[-1]
print("here")
while np.isclose(T, t_final) == True: 
    T = T * 10000
    ev_alpha_DF = inspiral.Classic.Evolve(sp_b_alpha, a0, e0, a_fin = R_fin, t_fin = T, opt=opt_DF)
    t_final = ev_alpha_DF.t[-1]


print("t_final =", t_final/year_to_pc, "yr")

a_i =  0.0007451535500315954 pc
Evolving from  2587.338715387484  to  1.0 r_isco  with initial eccentricity 0.999999499999875  with  Options: dissipative Forces emplyed {GasDynamicalFriction, GWLoss, }, accuracy = 1.0e-13


/home/pratibhajangra/Downloads/imripy-main/src/imripy/inspiral/inspiral.py:329: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  t_coal = t_coal * 48./19. / g(e_0)**4 * quad(lambda e: g(e)**4 *(1-e**2)**(5./2.) /e/(1. + 121./304. * e**2), 0., e_0, limit=100)[0]   # The inspiral time according to Maggiore (2007)
/home/pratibhajangra/Downloads/imripy-main/src/imripy/inspiral/forces.py:314: RuntimeWarning: invalid value encountered in log
  1./2. * np.log((1. + np.abs(v_rel)/c_s)/(1. - np.abs(v_rel)/c_s)) - np.abs(v_rel)/c_s) # subsonic regime
/home/pratibhajangra/Downloads/imripy-main/src/imripy/inspiral/forces.py:313: RuntimeWarning: invalid value encountered in log
  1./2. * np.log(1. - (c_s/np.abs(v_rel))**2) + ln_Lambda, # supersonic regime


A termination event occurred.
 -> Evolution took 65.4042s
here
t_final = 3.1481368941495394e-09 yr


In [ ]:
plt.figure(figsize = (10, 5))
plt.rc('lines', linewidth = 1)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.tick_params(which='both', right=True, top=True)

# Setting initial and final conditions of the binary evolution for an eccentric orbit
j0_array = np.geomspace(1e-3, 1, 5)
j0_ref_list = j0_array.tolist()
t_final_list = np.zeros(len(j0_ref_list))
T_list = np.zeros(len(j0_ref_list))



#system with m_1 = 1e5 solar mass and m_2 = 1 solar mass.
for k, j0 in enumerate (j0_ref_list):
        m1 =  1e5 * solar_mass_to_pc  
        m2 = 1 * solar_mass_to_pc  
        D = 0
        accuracy = 1e-13
        alpha = 0.1
        f_edd = 0.1
        eps = 0.1
        sp_0 = ms.SystemProp(m1, m2, halo.ConstHalo(0.), D)
        alphaDisk = halo.AlphaDisk(m1, alpha, f_edd, eps)
        sp_b_alpha = ms.SystemProp(m1, m2, halo.ConstHalo(0.), baryonicHalo = alphaDisk, D=D)
        opt_DF = inspiral.Classic.EvolutionOptions(dissipativeForces={forces.GWLoss(), forces.GasDynamicalFriction()}, 
                                                 considerRelativeVelocities=True, progradeRotation=False, accuracy=accuracy, verbose=1)



        a0 = a_i * 0.1
        print("a_initial =", a0, "pc")
        e0 = np.sqrt(1 -(j0**2))
        R_fin = sp_0.r_isco()   #Choosen equal to r_icso, The final condition for the evolution



        m1_ref1 = 1e-2          # in units of solar mass
        m1_ref2 = 1             # in units of solar mass
        T = 1e-6 * (t_m * year_to_pc ) * ((m1_ref1/m1_ref2)**(-1))

        ev_alpha_DF = inspiral.Classic.Evolve(sp_b_alpha, a0, e0, a_fin = R_fin, t_fin = T, opt=opt_DF)
        t_final = ev_alpha_DF.t[-1]
        print("here")
        while np.isclose(T, t_final) == True: 
            T = T * 10000
            ev_alpha_DF = inspiral.Classic.Evolve(sp_b_alpha, a0, e0, a_fin = R_fin, t_fin = T, opt=opt_DF)
            t_final = ev_alpha_DF.t[-1]

        t_final_list[k] = t_final/year_to_pc
        T_list[k] =  T/year_to_pc
      
    #np.savez("tmofjo_multipeak_1",  j0_ref_list_1,  tm_ref_list_1)

    
    
plt.loglog(j0_ref_list, t_final_list, '--o')
print("t_final_list =", t_final_list, "yr")
    

plt.xlabel('$j_{i}$', fontsize = 13)
plt.ylabel('final merger time / yr', fontsize = 12)
plt.title('$m_{1} = 10^{5} \, M_{\odot}$ and $m_{2}= 1 \,  M_{\odot}$ ')
plt.grid()
plt.savefig('merger_time1.pdf', bbox_inches = 'tight')
plt.show()

a_initial = 7.451535500315955e-05 pc
Evolving from  2587.338715387484  to  1.0 r_isco  with initial eccentricity 0.999999499999875  with  Options: dissipative Forces emplyed {GasDynamicalFriction, GWLoss, }, accuracy = 1.0e-13
